# Introduction to tf-shell

To get started, `pip install tf-shell`. tf-shell has a few modules, the one used
in this notebook is `tf_shell`.

In [1]:
import tf_shell
import tensorflow as tf
import timeit

context = tf_shell.create_context64(
    log_n=10,
    main_moduli=[8556589057, 8388812801],
    plaintext_modulus=40961,
    scaling_factor=3,
    mul_depth_supported=3,
    seed="test_seed",
)

secret_key = tf_shell.create_key64(context)
rotation_key = tf_shell.create_rotation_key64(context, secret_key)

a = tf.random.uniform([context.num_slots, 55555], dtype=tf.float32, maxval=10)
b = tf.random.uniform([55555, 333], dtype=tf.float32, maxval=10)
c = tf.random.uniform([2, context.num_slots], dtype=tf.float32, maxval=10)
d = tf.random.uniform([context.num_slots, 4444], dtype=tf.float32, maxval=10)

enc_a = tf_shell.to_encrypted(a, secret_key, context)

2024-06-10 21:30:04.256734: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-06-10 21:30:04.257664: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-06-10 21:30:04.291533: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-06-10 21:30:04.428601: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-10 21:30:05.195988: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF

In [2]:
def to_pt():
    return tf_shell.to_shell_plaintext(a, context)

time = min(timeit.Timer(to_pt).repeat(repeat=3, number=1))
print(time)

0.4906675929996709


In [3]:
def enc():
    return tf_shell.to_encrypted(d, secret_key, context)

time = min(timeit.Timer(enc).repeat(repeat=3, number=1))
print(time)

5.263423050000711


In [4]:
def dec():
    return tf_shell.to_tensorflow(enc_a, secret_key)

time = min(timeit.Timer(dec).repeat(repeat=3, number=1))
print(time)

0.5277276859997073


In [5]:
def ct_ct_add():
    return enc_a + enc_a

time = min(timeit.Timer(ct_ct_add).repeat(repeat=3, number=1))
print(time)

0.4192462440005329


In [6]:
def ct_ct_sub():
    return enc_a - enc_a

time = min(timeit.Timer(ct_ct_sub).repeat(repeat=3, number=1))
print(time)

0.4219015720009338


In [7]:
def ct_ct_mul():
    return enc_a * enc_a

time = min(timeit.Timer(ct_ct_mul).repeat(repeat=3, number=1))
print(time)

0.8668678089998139


In [8]:
def ct_pt_add():
    return enc_a + a

time = min(timeit.Timer(ct_pt_add).repeat(repeat=3, number=1))
print(time)

0.7579904609992809


In [9]:
def ct_pt_mul():
    return enc_a * a

time = min(timeit.Timer(ct_pt_mul).repeat(repeat=3, number=1))
print(time)

0.6268679120003071


In [10]:
def ct_pt_matmul():
    return tf_shell.matmul(enc_a, b)

time = min(timeit.Timer(ct_pt_matmul).repeat(repeat=3, number=1))
print(time)

25.57404864599812


In [11]:
def pt_ct_matmul():
    return tf_shell.matmul(c, enc_a, rotation_key)

time = min(timeit.Timer(pt_ct_matmul).repeat(repeat=3, number=1))
print(time)

361.1888753159983


In [12]:
def ct_roll():
    return tf_shell.roll(enc_a, 2, rotation_key)

time = min(timeit.Timer(ct_roll).repeat(repeat=3, number=1))
print(time)

4.650902364999638
